In [1]:
%run setup_levy_model_posner.py

2024-06-13 08:47:37.743765: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 08:47:37.745379: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 08:47:37.764725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 08:47:37.764742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 08:47:37.765242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2.15.1


INFO:root:Performing 2 pilot runs with the lfm model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 7)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 600, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


[[1.1 1.1 3.  3.  0.3 0.2 1.5]] [[0.6 0.6 3.  3.  0.2 0.2 0.3]]


INFO:root:Loaded loss history from checkpoints//posner/history_100.pkl.
INFO:root:Networks loaded from checkpoints//posner/ckpt-100
INFO:root:Performing a consistency check with provided components...
INFO:root:Done.


In [2]:
# Loading data
import pandas as pd
import os
import fnmatch
import glob 
from keras.utils import to_categorical

In [3]:
def config_observed_data(forward_dict):
    """Configure the output of the GenerativeModel for a BayesFlow setup."""

    # Prepare placeholder dict
    out_dict = {}

    # Extract simulated response times
    data = forward_dict["data"]
    data = data[np.newaxis,:,:]
    out_dict["summary_conditions"] = data.astype(np.float32)

    # Make inference network aware of varying numbers of trials
    # We create a vector of shape (batch_size, 1) by repeating the sqrt(num_obs)
    vec_num_obs = np.sqrt(data.shape[0]) * np.ones((data.shape[0], 1))
    out_dict["direct_conditions"] = np.sqrt(vec_num_obs).astype(np.float32)

    return out_dict

In [12]:
def get_posterior_means(file):
    raw_data = pd.read_csv(file)
    # raw_data["stim"] = raw_data["stim"]-1
    # raw_data["rt"] = raw_data["rt"]/1000
    raw_data["rt"] = (raw_data["resp"]*2 - 1) * raw_data["rt"] # to negate wrong rts
    
    data = np.c_[raw_data["condition"], raw_data["rt"].to_numpy()]
    input_dict = {
        'data': data
    }
    
    theta_hat = amortizer.sample(config_observed_data(input_dict), 5000, to_numpy=True)
    std_means = np.mean(theta_hat, axis=0)
    return(prior_means + std_means*prior_stds)

In [14]:
# Loop through each datafile and estimate 
path_to_data = 'data/levy_data/posner'
PAR = pd.DataFrame(columns=['sub', 'a_pi', 'a_ni', 'v_pi', 'v_ni', 't', 'st', 'alpha'])

mask = path_to_data + '/posner_data*.csv'
datafiles = glob.glob(mask) 

for file in datafiles:
    print(file)
    par = get_posterior_means(file)
    print(np.round(par,2))
    result = pd.DataFrame(par.reshape(1,7),columns=['a_pi', 'a_ni', 'v_pi', 'v_ni', 't', 'st', 'alpha'])
    result['sub'] = file.split('_')[-1].split('.')[0]
    # result['task'] = task
    PAR = pd.concat([PAR, result], ignore_index=True)    

data/levy_data/posner/posner_data_subject_1.csv
[[2.39 2.47 4.08 3.58 0.44 0.13 1.99]]
data/levy_data/posner/posner_data_subject_10.csv


/tmp/ipykernel_9373/1458937407.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  PAR = pd.concat([PAR, result], ignore_index=True)


[[2.39 2.62 3.83 3.2  0.46 0.09 2.  ]]
data/levy_data/posner/posner_data_subject_100.csv
[[2.09 2.38 5.1  3.95 0.59 0.14 2.02]]
data/levy_data/posner/posner_data_subject_101.csv
[[1.46 2.21 5.99 6.4  0.47 0.1  2.  ]]
data/levy_data/posner/posner_data_subject_102.csv
[[2.29 2.53 4.06 4.1  0.48 0.12 2.01]]
data/levy_data/posner/posner_data_subject_103.csv
[[2.37 2.41 3.7  3.88 0.42 0.1  1.98]]
data/levy_data/posner/posner_data_subject_104.csv
[[1.91 1.96 4.51 4.96 0.45 0.08 1.93]]
data/levy_data/posner/posner_data_subject_105.csv
[[2.45 2.3  3.75 3.2  0.43 0.05 2.02]]
data/levy_data/posner/posner_data_subject_106.csv
[[3.29 4.26 5.03 4.06 0.4  0.44 1.84]]
data/levy_data/posner/posner_data_subject_107.csv
[[2.18 2.34 4.27 5.2  0.44 0.08 2.04]]
data/levy_data/posner/posner_data_subject_108.csv
[[2.22 2.57 3.75 4.07 0.4  0.1  2.  ]]
data/levy_data/posner/posner_data_subject_109.csv
[[1.8  2.15 3.18 4.65 0.36 0.07 1.88]]
data/levy_data/posner/posner_data_subject_11.csv
[[2.3  2.73 3.36 4.19 

In [15]:
PAR.to_csv(path_to_data + '/levy_mat_posner.csv', index=False)